### レシピ

- 来場者予測モデルの構築
- XGBRegressorアルゴリズムを利用した学習と予測
- 決定係数を用いたモデルの評価

#### レシピの概要
- このレシピでは、コワーキングスペースの来館者予測モデルを構築していきます。最終的には小売店や飲食店などの売上予測モデルもこのモデルを基礎として構築することができます。
- 今回はコワーキングスペースの過去半年分のデータを元に来館者予測モデルを構築していきます。流れとしてはデータの読み込み→データの加工→データの分割→アルゴリズムによる予測モデルの構築→評価となっています。

#### 各種ライブラリのインポートと事前処理

In [7]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings("ignore")

# ライブラリの import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# Numpy の不動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandas での不動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
# データフレームで全ての項目を表示
pd.set_option("display.max_columns", None)
# グラフのデフォルトフォント設定
plt.rcParams["font.size"] = 14
# 乱数の値
random_seed = 123

#### データの読み込みとデータ前処理

In [9]:
# データの読み込み
data = pd.read_csv("./data/coworking.csv", parse_dates=[0])
# データ属性の確認
print(data.dtypes)

日付                datetime64[ns]
イベント数                      int64
会議室予約数                     int64
1Day利用者数                   int64
昼の天気                       int64
平均気温                     float64
曜日                         int64
月                          int64
コワーキングスペース入場者数             int64
dtype: object


In [11]:
# 日付データの変換
data["日付"] = pd.to_datetime(data["日付"])
data["日付"] = data["日付"].dt.strftime("%Y%m")

In [12]:
data.head(3)

,日付,イベント数,会議室予約数,1Day利用者数,昼の天気,平均気温,曜日,月,コワーキングスペース入場者数
0,202006,0,8,1,1,19.7000,0,6,50
1,202006,0,13,2,0,22.0000,1,6,41
2,202006,0,10,1,0,20.1000,2,6,49


今回の学習は「教師あり学習」に分類されます。入力データは

- 日付
- イベント数
- 会議室予約数
- 1Day利用者数
- 昼の天気
- 曜日
- 月

となります。そして

- コワーキング入場者数

を予測していきます。

In [14]:
# 項目名
columns = ['日付','イベント数','会議室予約数','1Day利用数','昼の天気','平均気温','曜日','月','コワーキング入場者数']
#項目名の置き換え
data.columns = columns

In [15]:
data.head()

,日付,イベント数,会議室予約数,1Day利用数,昼の天気,平均気温,曜日,月,コワーキング入場者数
0,202006,0,8,1,1,19.7000,0,6,50
1,202006,0,13,2,0,22.0000,1,6,41
2,202006,0,10,1,0,20.1000,2,6,49
3,202006,0,10,1,0,20.2000,3,6,38
4,202006,0,10,1,0,21.9000,4,6,43


In [17]:
# 欠損値チェック
data.isnull().sum()

日付            0
イベント数         0
会議室予約数        0
1Day利用数       0
昼の天気          0
平均気温          0
曜日            0
月             0
コワーキング入場者数    0
dtype: int64

In [19]:
# 今回の入力データでは不要の「日付」「コワーキングスペース入場者数」の削除を行う。説明変数。
X = data.drop(["日付", "コワーキング入場者数"], axis=1)

In [20]:
# 目的変数
y = data["コワーキング入場者数"].values

#### データの分割

訓練データと検証データの分割を行います。今回は訓練データと検証データを 8:2 の割合で分割を行います。

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=1)

# 結果確認
print(X_train.shape)
print(X_val.shape)

(146, 7)
(37, 7)


#### モデルの学習と予測

モデル作成のためのアルゴリズムを選択します。今回は XGBRegressor を活用してモデルを作っていきたいと思います。

In [23]:
from xgboost import XGBRegressor

algorithm = XGBRegressor(objective = 'reg:squarederror', random_state=random_seed)

In [24]:
# 学習
algorithm.fit(X_train, y_train)
# 予測
y_pred = algorithm.predict(X_val)
# 予測結果確認
print(y_pred[:5])

[58.3316 59.1509 36.8311 80.0591 42.4841]


#### モデルの評価

決定係数R2を用いてモデルの評価を行っていきます。決定係数R2は、０から１までの数字を取り、１に近づくほど精度の高い予測モデルが構築できたことになります。

In [26]:
# 評価
score = algorithm.score(X_val, y_val)

# R2値の計算
from sklearn.metrics import r2_score
r2_score = r2_score(y_val, y_pred)

# 結果確認
print(f'score: {score:.4f} r2_score: {r2_score:.4f}')

score: 0.6238 r2_score: 0.6238


決定係数は 0.6238 となりました。重回帰分析における決定係数は 0.5 以上で適合度の高いモデルと言えるので、今回構築したモデルは精度がある程度高いと結論付けられる。